# 新冠肺炎疫情分析


## 数据导入
- 使用2019.12.23-2020.2.21的微博热搜（top50）数据
    - 原排行榜小时级更新，为了便于分析，选取每天12:00的数据作为当天搜索排行榜数据

### 原始数据

- 微博热搜1223-0210.csv：原始小时级数据
   - 一共近8w条 (79278)，每条数据5个字段
   - 格式：【时间 排名 热搜内容 最后时间 上榜时间】

In [2]:
import pandas as pd
# 导入小时级数据
df_hour = pd.read_csv(open("微博热搜1223-0210.csv", encoding='utf8'), sep=',',dtype=str)
df1 = df_hour.filter(like='钟南山', axis=0)
#df1 = df_hour.filter(regex='12:00$', axis=0)
print(df_hour.size, df_hour.shape)
df_hour.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa7 in position 14: invalid start byte

### 疫情相关数据

- 原始数据是小时级，量大，而且很多热搜内容与疫情无关，不便分析，需要提炼疫情相关的内容，单独分析
- 微博热搜1.1-2.21.csv：缩减后的天级数据，仅保留疫情相关的搜索词，人工标注完成
    - 一共899条，数据量大规模减少，8w→900条
    - 格式【时间 排名 热搜内容 标注1 标注2 上榜时间 最后时间】
    - 注：
        - 标注1：地域信息，取值范围：全国数据、疫情地图、地区、新闻、人物、医院、物质、军队、新闻、知识、专家
        - 标注2：主题信息，取值范围：

In [ ]:
import datetime
#str_p = '2020/2/7 12:00'
#dateTime_p = datetime.datetime.strptime(str_p,'%Y/%m/%d %H:%M')

# 导入天级数据，1.30以后标注完整（地域+主题）
df_day = pd.read_csv(open("微博热搜1.1-2.21.csv", encoding='gbk'), sep=',',dtype=str)
# 时间格式矫正，规范，便于字符串排序
df_day['时间'] = df_day['时间'].apply(lambda x:str(datetime.datetime.strptime(x,'%Y/%m/%d %H:%M')).split(' ')[0])#.sort_index()
# 按照日期升序排列
df_day = df_day.sort_values(by='时间') 
print(df_day.size, df_day.shape)
df_day.head()

In [ ]:
df_day.sort_values(by='时间', ascending=False) # 逆序排列
df_day[df_day['排名'].isin(['1','2','3'])] # 提取每天的前三名
df_day[df_day['时间']=='2020-02-06'].head() # 查看某天的热门搜索

In [ ]:
import numpy as np

#df_new.groupby(['时间']).count().sort_values(by='时间')
df_day.groupby(['时间'])['排名'].agg([sum])
#df_day['时间'].groupby(['时间']).count().reset_index().rename(columns={'0':'数目'})
#df_day['排名'].astype(float).groupby(['时间']).size()
#df_day['排名'].astype(float).groupby(by=['时间'])
df_day.groupby('时间')['排名'].count().reset_index().rename(columns={'0':'数目'})
df_day.groupby('时间').count()[:2]

## 趋势分析
- 疫情相关热词近期有什么变化？

### 疫情热度如何变化？
- 每天疫情相关热门搜索词有多少？
    - 趋势图见末尾，分析如下：
    - 1.19前，只有1-2个热搜，此时疫情刚刚爆发，原因不明，只知道是冠状病毒，不确定是否人传人
    - 1.20开始，突然上涨到8个热搜，武汉新增136例，卫健委发布病毒性肺炎高发注意事项，同时疫情已经开始蔓延到省外：浙江发现5例、深圳出现8例（1例确诊）、北京确诊2例，上海也开始加强筛查
    - 1.21以后，武汉同济协和医院出现15名医护感染，钟南山站肯定新型冠状病毒肺炎人传人，民众开始高度关注，top 50榜单里至少一半是疫情相关
    - 1.23，武汉封城，31个热搜
    - 1.25，物资缺乏，除夕夜医护吃泡面，承受极大的压力，接近奔溃，大年初一，政府开始调度资源，医护、军队进入疫区
    - 1.26，榜单达到高峰：41个热搜，全国疫情迅速发展，民众焦虑，不知所措
    - 1.29，热词数目开始下滑，31个，小汤山医院投入使用，火神山马上完工，虽然各地确诊人数仍然不断增加，但大家没那么害怕
    - 2.6，小高峰，35个
    - 2.20，减少到15个
    - 其他分析。。。
- 每天疫情相关热门搜索词平均排名高吗？
    - 前期，热词少，突发情况排名较高，如武汉出现疫情
    - 中期，平均排名15-20，霸占榜单前几名
    - 后期，继续上升到8-10
- 每天关注度是如何变化？
    - >关注度 = 热门搜索排名累加

In [ ]:
import json

day_info_dict = {} # 趋势信息
loc_info_dict = {'total':{}, 'overseas':{}, 'prov':{}, 'city':{}} # 地域信息
#for i in df.iteritems(): # iterrows()
for i in df_day.iterrows():
    # 记录每天的搜索词数目
    (dt, rank, query, location, topic) = i[1]['时间'], float(i[1]['排名']), i[1]['热搜内容'], i[1]['地域'], i[1]['主题']
    if dt not in day_info_dict:
        # 记录排名信息，最大，最小，平均
        day_info_dict[dt] = {'query_num':1, 'min_rank':50-rank, 'max_rank':50-rank, 'avg_rank':50-rank}
    else:
        day_info_dict[dt]['query_num'] += 1 # 热词数目
        if rank < day_info_dict[dt]['min_rank']:
            day_info_dict[dt]['min_rank'] = 50-rank
        if rank > day_info_dict[dt]['max_rank']:
            day_info_dict[dt]['max_rank'] = 50-rank
        day_info_dict[dt]['avg_rank'] = round((day_info_dict[dt]['query_num']+day_info_dict[dt]['avg_rank']*(
            day_info_dict[dt]['query_num']-1))/day_info_dict[dt]['query_num'],2)
    # 地域信息 国内-湖北-武汉，国外-日本
    loc_list = location.split('-')
    loc_len = len(loc_list)
    if loc_len > 0: # 国内外信息
        area = loc_list[0]
        if area not in loc_info_dict['total']:
            loc_info_dict['total'][area] = [0, 0] # 频次、累计热度（50-名次）
        loc_info_dict['total'][area][0] += 1
        loc_info_dict['total'][area][1] += 50-rank # 排名越靠前，权重越大
    if loc_len > 1: # 省份信息、国家信息
        prov = loc_list[1].split('|')[0] # 复合形式：山东|浙江
        k = 'prov'
        if area == '国外':
            k = 'overseas'
        if prov not in loc_info_dict[k]:
            loc_info_dict[k][prov] = [0, 0, {}] # 频次、累计热度、城市信息
        loc_info_dict[k][prov][0] += 1
        loc_info_dict[k][prov][1] += 50-rank # 排名越靠前，权重越大
    if loc_len > 2: # 城市信息
        city =  loc_list[2]
        if city not in loc_info_dict[k][prov][2]:
            loc_info_dict[k][prov][2][city] = [0,0]
        loc_info_dict[k][prov][2][city][0] += 1
        loc_info_dict[k][prov][2][city][1] += 50-rank
        # 所有城市
        if city not in loc_info_dict['city']:
            loc_info_dict['city'][city] = [0,0]
        loc_info_dict['city'][city][0] += 1
        loc_info_dict['city'][city][1] += 50-rank
#print(day_info_dict)
out_dict = {'趋势':{'日期':[], '热词数目':[], '最大关注度':[], '最小关注度':[], '平均关注度':[]},
            '地域':{}}
out_dict['趋势']['日期'] = list(day_info_dict.keys())
out_dict['趋势']['热词数目'] = [day_info_dict[dt]['query_num'] for dt in out_dict['趋势']['日期']]
out_dict['趋势']['最大关注度'] = [day_info_dict[dt]['max_rank'] for dt in out_dict['趋势']['日期']]
out_dict['趋势']['最小关注度'] = [day_info_dict[dt]['min_rank'] for dt in out_dict['趋势']['日期']]
out_dict['趋势']['平均关注度'] = [day_info_dict[dt]['avg_rank'] for dt in out_dict['趋势']['日期']]
#print(json.dumps(out_dict, ensure_ascii=False))
tmp_df = pd.DataFrame(out_dict['趋势'])
tmp_df[:10]
# 世界地图只支持英文，需要先翻译过去
country_dict = {'泰国':'Thailand', '日本':'Japan', '美国':'United States', '英国':'United Kingdom', '韩国':'Korea', '新加坡':'Singapore', '法国':'French',
                '德国':'German', '巴西':'Brazil', '澳大利亚':'Australian', '芬兰':'Finland', '意大利':'Italian', '俄罗斯':'Russia', 
                '伊拉克':'Iraq', '伊朗':'Iran', '叙利亚':'Syria', '以色列':'Israel','巴勒斯坦':'Palestine','蒙古':'Mongolia','朝鲜':'Dem. Rep. Korea',
                '印度':'Indian', '阿富汗':'Afghanistan', '加拿大':'Canadia', '中国':'China'}
out_dict['地域']['总体'] = loc_info_dict['total']
out_dict['地域']['世界'] = {
    '国家':[country_dict[k] for k,v in loc_info_dict['overseas'].items() if k in country_dict],
    '频次':[v[0] for k,v in loc_info_dict['overseas'].items() if k in country_dict],
    '关注度':[v[1] for k,v in loc_info_dict['overseas'].items() if k in country_dict],
}
# 补充中国总体信息
out_dict['地域']['世界']['国家'].append('China')
out_dict['地域']['世界']['频次'].append(loc_info_dict['total']['国内'][0])
out_dict['地域']['世界']['关注度'].append(loc_info_dict['total']['国内'][1])
# 省份信息
out_dict['地域']['中国'] = {
    '省份':[k for k,v in loc_info_dict['prov'].items()],
    '频次':[v[0] for k,v in loc_info_dict['prov'].items()],
    '关注度':[v[1] for k,v in loc_info_dict['prov'].items()],
}
# 湖北城市信息
out_dict['地域']['湖北'] = {
    '城市':[k for k,v in loc_info_dict['prov']['湖北'][2].items()],
    '频次':[v[0] for k,v in loc_info_dict['prov']['湖北'][2].items()],
    '关注度':[v[1] for k,v in loc_info_dict['prov']['湖北'][2].items()],
}
# 全国城市信息
out_dict['地域']['城市'] = {
    '城市':[k for k,v in loc_info_dict['city'].items()],
    '频次':[v[0] for k,v in loc_info_dict['city'].items()],
    '关注度':[v[1] for k,v in loc_info_dict['city'].items()],
}

print(loc_info_dict['total'])
print(loc_info_dict['prov'])
print(loc_info_dict['overseas'])
print(loc_info_dict['city'])

## 主题分析
- 热搜内容分布在不同主题里
    - 疫情数据：全国确诊人数、疫情地图
    - 医护
    - 专家
    - 政府措施
    - 其他


- 以全国确诊人数为例，提取“全国累计确诊”前缀的热搜

In [ ]:
import re

df_c_total = df_day[df_day['热搜内容'].str.contains('全国累计确诊*')] # 字符串匹配，查找类似搜索
# 提取热搜里的数字
pattern_num = re.compile(r'[\D+]*(\d+).*')

def get_number(text):
    """ 提取数字 """
    res = pattern_num.match(text)
    if not res:
        return 0
    return res.groups()[0]
total_num_dict = {}
# for i in df_c_total.iterrows():
#     # 记录每天的搜索词数目
#     (dt, rank, query) = i[1]['时间'].split()[0], float(i[1]['排名']), i[1]['热搜内容']
df_c_total['确诊人数'] = df_c_total['热搜内容'].apply(get_number)
out_dict['确诊'] = {'日期':list(df_c_total['时间']), '排名':list(df_c_total['排名']), '确诊人数':list(df_c_total['确诊人数'])}
#out_dict['确诊']
df_c_total.head()

In [ ]:
df_c_map = df_day[df_day['热搜内容'].str.contains('地图')] # 字符串匹配，查找类似搜索
#df_c_map
out_dict['疫情地图'] = {'日期':list(df_c_map['时间']), '关注度':list(df_c_map['排名'])}
df_c_map.head()

In [ ]:
df_day[df_day['热搜内容'].str.contains('[医生|护士]')].head()

## 疫情内容分析
- 采用NLP技术肢解热门搜索词，尝试挖掘内部隐藏的信息
    - 参考：[Python的jieba分词及TF-IDF和TextRank 算法提取关键字](https://blog.csdn.net/bozhanggu2239/article/details/80157305)

In [ ]:
#import requests
import pandas as pd
import datetime
import re
import jieba
from jieba import analyse
import jieba.posseg as pseg
#import matplotlib.pyplot as plt
#import jieba.analyse

# 分词
# sent_words = [list(jieba.cut(sent0)) for sent0 in sentences]
# document = [" ".join(sent0) for sent0 in sent_words]
# print(document)
# 自定义词库
#jieba.set_dictionary(file_name)
#jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径
entity_list = ['新型冠状病毒/8/nz', '冠状病毒/8/nz','疫情地图/8/nz','武汉肺炎/8/nz','人传人','病毒/8/nz','疫情/8/nz',
               '病例','新冠/8/nz', 'SARS/8/nz','隔离', '口罩', '卫健委/5/nt','双黄连/8/nz','埃博拉/3/nz','瑞德西韦/5/nz',
           '金银潭/5/nz','百步亭/5/nz','火神山/5/nz','雷神山/5/nz','张家界/5/ns', '小汤山/5/nz','威斯特丹号/5/nz','钻石公主号/5/nz',
               '方舱医院/5/nz','同济医院/3/nz','协和医院/3/nz','紫外线', '蝙蝠/3/nz','酒精','消毒','确诊','连闯/2/v',
               '康复/3/n','封城/3/n','汤圆/3/n', '武软/3/nt', '红十字会/3/nt', '马拉松/3/n', '小姐姐/3/r',
               '医护人员/5/nr','医疗队/5/nr','医院院长/5/nr','张定宇/5/nr','护士/5/nr','医生/5/nr',
               '钟南山/5/nr', '李兰娟/5/nr','李文亮/5/nr', '应勇/5/nr','王贺胜/5/nr', '王忠林/5/nr']
for w in entity_list:
    #print(w)
    w_list = w.split('/')
    freq, tag = 1, None
    if len(w_list) > 1:
        # [word freq]
        freq = w_list[1]
    if len(w_list) > 2:
        # [word freq tag]
        tag = w_list[2]
    #jieba.add_word(w)
    jieba.add_word(w_list[0], freq=freq, tag=tag)

word_list = []
for index, row in df_day.iterrows():
    content = row[2]
    #TextRank 关键词抽取，只获取固定词性
    #words = analyse.textrank(content, topK=50, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))
    #words = analyse.textrank(content, topK=50)
    # res = analyse.extract_tags(text) # TF-IDF关键词提取
    # 不用关键词工具，因为无法导入自定义词库
    for w in pseg.cut(content):
        if not ( w.flag.startswith('n') or w.flag in ('v','vn','x')):
            continue
        if len(w.word) == 1:
            continue
        # 记录全局分词
        word_list.append({'word':w.word,'pos':w.flag, 'count':1})
df_kw = pd.DataFrame(word_list)
# 词频统计
df_kw = df_kw.groupby(['word','pos'])['count'].sum().reset_index().sort_values(by='count', ascending=False)
df_kw.head()

In [ ]:
df_day[df_day['热搜内容'].str.contains('连闯')]

In [ ]:
print('\t'.join(df_kw['word'][:100]))
df_day[df_day['热搜内容'].str.contains('连闯')]
#df_kw[df_kw['word'].str.contains('小汤山')]
# 关键词分门别类
tag_dict = {'n':'名词', 'nr':'人物', 'ns':'地名','nz':'专名', 'nt':'机构组织', 'v':'动词', 'vn':'动作'}
# 词性：n,v,ns,nr,vn,nt,nz,x,ng,nrt
df_kw[df_kw.pos=='nr'][:50] # 人名
#df_kw[df_kw.pos=='ns'][:50] # 各地地名
#df_kw[df_kw.pos=='n'][:50] # 名词
#df_kw[df_kw.pos=='v'][:50] # 动词
#df_kw.groupby(['pos']).count().sort_values(by='count', ascending=False)
df_kw[df_kw.pos=='vn'][:50] # 动作
df_kw[df_kw.pos=='nt'][:50] # 各机构
out_dict['关键词'] = {}
for tag in tag_dict:
    out_dict['关键词'][tag] = {'含义': tag_dict[tag], 
                '数据':df_kw[['word','count']][df_kw.pos==tag].values.tolist()}
#out_dict['关键词']

In [ ]:
import jieba.posseg as pseg
#df_kw
content = '钟南山承认人传人现象，李文亮说了'
words = analyse.textrank(content, topK=50) 
words = analyse.extract_tags(content, topK=50)
print(words)
pos = [ '{}/{}'.format(w.word,w.flag) for w in pseg.cut(content) if w.flag.startswith('n') or w.flag in ('v','vn','x')]
print('\t'.join(pos))
print('\t'.join(df_kw[df_kw.pos=='nr']['word']))
#df_kw[['word','count']][df_kw.pos=='nr'][:100].values.tolist()

## 可视化

In [ ]:
df_day[df_day['热搜内容'].str.contains('连闯')]
df_day.values.tolist()
#list(zip(out_dict['地域']['中国']['省份'], out_dict['地域']['中国']['关注度']))
[ list(a) for a in zip(out_dict['地域']['中国']['省份'], out_dict['地域']['中国']['关注度'])][:28]

In [ ]:
#先安装pip install pyecharts==0.5.1
#from IPython.display import HTML, SVG
# from IPython.core.magic import register_cell_magic
import pyecharts as pe

page = pe.charts.Page('疫情搜索词分析')
opts = pe.options # 配置信息
t1 = '疫情搜索词分析（每天12:00的数据）'
s1 = '2020-2-24'
# 图像数据
image = pe.components.Image()
img_src = (
    "http://inews.gtimg.com/newsapp_match/0/11297270903/0"
    #"http://pics1.baidu.com/feed/72f082025aafa40fb2156a3decbe23497af0195b.jpeg"
)
image.add(
    src=img_src,
    style_opts={"width": "60%", "height": "100%", "style": "margin-top: 20px"},
).set_global_opts(
    title_opts=pe.options.ComponentTitleOpts(title="武汉加油", subtitle=s1)
)
page.add(image)
# 表格数据
table = pe.components.Table()
table.add(list(df_day.columns), df_day.values.tolist()[:20])
table.set_global_opts(title_opts=pe.options.ComponentTitleOpts(title=t1+'(表格)', subtitle=s1))
page.add(table)
# (1) 每天热搜数目排名变化
line_trend = pe.charts.Line()
line_trend.set_global_opts(title_opts=opts.TitleOpts(title='疫情热搜趋势分析',pos_left='center',pos_top='8%',subtitle=s1),
                          # legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle',orient='vertical'), # 靠左居中垂直排列
                      toolbox_opts=opts.ToolboxOpts())
line_trend.add_xaxis(out_dict['趋势']['日期'])\
    .add_yaxis('疫情热搜-数目', out_dict['趋势']['热词数目'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5),)\
    .add_yaxis('疫情热搜-最大关注度（50-rank）', out_dict['趋势']['最大关注度'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5))\
    .add_yaxis('疫情热搜-最小关注度（50-rank）', out_dict['趋势']['最小关注度'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5))\
    .add_yaxis('疫情热搜-平均关注度（50-rank）', out_dict['趋势']['平均关注度'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5))
#page.add(line)
bar = pe.charts.Bar()
bar.set_global_opts(title_opts=opts.TitleOpts(title='疫情热搜趋势分析',pos_left='center',subtitle=s1),
                    legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle',orient='vertical'), # 靠左居中垂直排列
                      toolbox_opts=opts.ToolboxOpts())
bar.add_xaxis(out_dict['趋势']['日期'])\
    .add_yaxis('疫情热搜-数目', out_dict['趋势']['热词数目'],
               label_opts=opts.LabelOpts(is_show=False)) # 隐藏标签，避免重影
bar.overlap(line_trend)
#page.add(bar)
page.add(line_trend)
# 确诊人数及疫情地图分析
line = pe.charts.Line()
line.set_global_opts(title_opts=opts.TitleOpts(title='疫情热搜-确诊人数变化',pos_left='center',pos_top='8%',subtitle=s1),
                    # legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle',orient='vertical'), # 靠左居中垂直排列
                      toolbox_opts=opts.ToolboxOpts())
line.add_xaxis(out_dict['确诊']['日期'])\
    .add_yaxis('确诊人数', out_dict['确诊']['确诊人数'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5))
page.add(line)
# 排名分析--确诊人数
line = pe.charts.Line()
line.set_global_opts(title_opts=opts.TitleOpts(title='疫情热搜关注度分析',pos_left='center',pos_top='8%',subtitle=s1),
                    # legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle',orient='vertical'), # 靠左居中垂直排列
                      toolbox_opts=opts.ToolboxOpts())
line.add_xaxis(out_dict['确诊']['日期'])\
    .add_yaxis('疫情热搜-确诊人数', out_dict['确诊']['排名'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5))
# 排名分析--疫情地图
line1 = pe.charts.Line()
line1.set_global_opts(title_opts=opts.TitleOpts(title='疫情热搜关注度分析',subtitle=s1),
                      #legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle',orient='vertical'), # 靠左居中垂直排列
                      toolbox_opts=opts.ToolboxOpts())
line1.add_xaxis(out_dict['疫情地图']['日期'])\
    .add_yaxis('疫情热搜-疫情地图', out_dict['疫情地图']['关注度'], linestyle_opts=opts.LineStyleOpts(width=5, opacity=0.5))
line.overlap(line1)
page.add(line)
# 地图可视化
#   世界地图
for k in ('频次', '关注度'):
    map_p = pe.charts.Map()
    num_range = (min(out_dict['地域']['中国'][k]), sorted(out_dict['地域']['中国'][k])[-2])
    t = "(国内频次：{}, 关注度:{}，国外频次：{}，, 关注度:{})".format(
                                out_dict['地域']['总体']['国内'][0], out_dict['地域']['总体']['国内'][1],
                                out_dict['地域']['总体']['国外'][0], out_dict['地域']['总体']['国外'][1])
    map_p.set_global_opts(title_opts=opts.TitleOpts(
                            title="新冠肺炎疫情全球分布-{}".format(k),
                            pos_left='center',subtitle='微博热搜榜单-{}'.format(t)), 
                          legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle',orient='vertical'), # 靠左居中垂直排列
                          toolbox_opts=opts.ToolboxOpts(),
                          visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1]))
    map_p.add(k, zip(out_dict['地域']['世界']['国家'], out_dict['地域']['世界'][k]), maptype="world")
    map_p.set_series_opts(label_opts=opts.LabelOpts(is_show=False)) # 世界地图不显示label
    #map_p.add('关注度', zip(out_dict['地域']['中国']['省份'], out_dict['地域']['中国']['关注度']), maptype="china")
    page.add(map_p)

# 中国地图
for k in ('频次', '关注度'):
    map_p = pe.charts.Map()
    num_range = (min(out_dict['地域']['中国'][k]), sorted(out_dict['地域']['中国'][k])[-2])
    t = "(国内频次：{}, 关注度:{}，国外频次：{}，, 关注度:{})".format(
                                out_dict['地域']['总体']['国内'][0], out_dict['地域']['总体']['国内'][1],
                                out_dict['地域']['总体']['国外'][0], out_dict['地域']['总体']['国外'][1])
    map_p.set_global_opts(title_opts=opts.TitleOpts(
                            title="新冠肺炎疫情全国分布-{}".format(k),
                            pos_left='center',subtitle='微博热搜榜单-{}'.format(t)), 
                          legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle'),
                          toolbox_opts=opts.ToolboxOpts(),
                          visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1]))
    map_p.add(k, zip(out_dict['地域']['中国']['省份'], out_dict['地域']['中国'][k]), maptype="china")
    #map_p.add('关注度', zip(out_dict['地域']['中国']['省份'], out_dict['地域']['中国']['关注度']), maptype="china")
    page.add(map_p)
    # 热力图
    geo = pe.charts.Geo()
    geo.add_schema(maptype="china")
    geo.add(k, list(zip(out_dict['地域']['中国']['省份'], out_dict['地域']['中国'][k])),type_=pe.globals.ChartType.HEATMAP)
    geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="各省份关注度热力图"),
    )
    page.add(geo)
# 湖北省内城市地图
for k in ('频次', '关注度'):
    map_hubei = pe.charts.Map()
    num_range = (min(out_dict['地域']['湖北'][k]), max(out_dict['地域']['湖北'][k]))
    map_hubei.set_global_opts(title_opts=opts.TitleOpts(
                                title="新冠肺炎疫情（湖北省内）-{}".format(k),
                                pos_left='center',subtitle='微博热搜榜单'), 
                              legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle'),
                          toolbox_opts=opts.ToolboxOpts(),
                          visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1]))
    city_name = ['{}市'.format(i) for i in out_dict['地域']['湖北']['城市']]
    map_hubei.add(k, zip(city_name, out_dict['地域']['湖北'][k]), maptype='湖北')
    #map_hubei.add('关注度', zip(city_name, out_dict['地域']['湖北']['关注度']), maptype='湖北')
    page.add(map_hubei)
# 全国城市信息 loc_info_dict['prov']
for k in ('频次', '关注度'):
    # 区块图（全国城市绘制无效）
#     map_hubei = pe.charts.Map()
#     num_range = (min(out_dict['地域']['城市'][k]), max(out_dict['地域']['城市'][k]))
#     map_hubei.set_global_opts(title_opts=opts.TitleOpts(
#                                 title="新冠肺炎疫情（全国）-{}".format(k),
#                                 pos_left='center',subtitle='微博热搜榜单'), 
#                               legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle'),
#                           toolbox_opts=opts.ToolboxOpts(),
#                           visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1]))
#     city_name = ['{}市'.format(i) for i in out_dict['地域']['城市']['城市']]
#     map_hubei.add(k, zip(city_name, out_dict['地域']['城市'][k]), maptype='china')
#     #map_hubei.add('关注度', zip(city_name, out_dict['地域']['湖北']['关注度']), maptype='湖北')
#     page.add(map_hubei)
    # 热力图
    geo = pe.charts.Geo()
    geo.add_schema(maptype="china")
    geo.add(k, list(zip(out_dict['地域']['城市']['城市'], out_dict['地域']['城市'][k])),type_=pe.globals.ChartType.HEATMAP)
    geo.set_series_opts(label_opts=opts.LabelOpts(is_show=False)).set_global_opts(
        visualmap_opts=opts.VisualMapOpts(),
        title_opts=opts.TitleOpts(title="全国城市热力图-{}".format(k)),
    )
    page.add(geo)

# (2) 标题关键词词云
#-饼图
top_n = 30
pie = pe.charts.Pie()
pie.set_global_opts(title_opts=opts.TitleOpts(title='疫情热词分析-饼图',pos_left='center',subtitle=s1),
                    legend_opts=opts.LegendOpts(is_show=False),
                    #legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle'),
                      toolbox_opts=opts.ToolboxOpts())
pie.add("饼图可视化（Top {}）".format(top_n), df_kw[['word','count']].values.tolist()[:top_n])
page.add(pie)

# 关键词词云可视化
wc = pe.charts.WordCloud()
wc.set_global_opts(title_opts=opts.TitleOpts(title='疫情热词分析-{}'.format('全部关键词'),
                                            pos_left='center',subtitle=s1))
wc.add("疫情热词分析-所有关键词", df_kw[['word','count']].values.tolist(), shape='circle') # word_size_range=[10, 500]
page.add(wc)
# 显示各类词云
for tag in tag_dict:
    wc = pe.charts.WordCloud()
    wc.set_global_opts(title_opts=opts.TitleOpts(title='疫情热词分析-{}'.format(out_dict['关键词'][tag]['含义']),
                                                 pos_left='center',subtitle=s1),
                       legend_opts=opts.LegendOpts(pos_left='left',pos_top='middle'),
                      toolbox_opts=opts.ToolboxOpts())
    wc.add(out_dict['关键词'][tag]['含义'], 
           out_dict['关键词'][tag]['数据'], shape='circle') # word_size_range=[10, 500]
    page.add(wc)

# 统一绘制、落地
out_file = 'wqw_sari.html'
page.render(out_file)
#page.render_notebook()
#HTML(out_file)

In [ ]:

print(out_dict['地域']['世界'])
value = [95.1, 23.2, 43.3, 66.4, 88.5]
attr = ["China", "Canada", "Brazil", "Russia", "United States"]
map0 = pe.charts.Map()
map0.add("世界地图", zip(attr, value), maptype="world")
#map0.render(path="世界地图.html")
#map0.render_notebook()

In [ ]:
map_hubei = pe.charts.Map()
num_range = (min(out_dict['地域']['湖北']['频次']), max(out_dict['地域']['湖北']['频次']))
map_hubei.set_global_opts(title_opts=opts.TitleOpts(
                        title="新冠肺炎疫情（湖北省内）",
                        subtitle='微博热搜榜单'), 
                      toolbox_opts=opts.ToolboxOpts(),
                      visualmap_opts=opts.VisualMapOpts(min_=num_range[0], max_=num_range[1]))
city_name = ['{}市'.format(i) for i in out_dict['地域']['湖北']['城市']]
print(city_name)
print(out_dict['地域']['湖北']['频次'])
print(out_dict['地域']['湖北']['关注度'])
map_hubei.add('频次', zip(city_name, out_dict['地域']['湖北']['频次']), maptype='湖北')
map_hubei.add('关注度', zip(city_name, out_dict['地域']['湖北']['关注度']), maptype='湖北')
#map_hubei.render_notebook()